In [47]:
import pandas as pd
import numpy as np
import os
import sqlalchemy
import sqlite3 as sql
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

#os.listdir("../KAGGLE/DATA")

database = "../KAGGLE_PRUBAS_DATA/DATA/diamonds_train.db"
connection = sql.connect(database)

query = '''SELECT tr.index_id, c.clarity, cl.color, ct.cut, dm.depth, dm.'table', dm.x, dm.y, dm.z, tr.price, ci.city, tr.carat
FROM diamonds_properties pr
   INNER JOIN diamonds_clarity c ON pr.clarity_id = c.clarity_id
   INNER JOIN diamonds_color cl on pr.color_id = cl.color_id
   INNER JOIN diamonds_cut ct on pr.cut_id = ct.cut_id
   INNER JOIN diamonds_dimensions dm on pr.index_id = dm.index_id
   INNER JOIN diamonds_transactional tr on pr.index_id = tr.index_id
   INNER JOIN diamonds_city ci on tr.city_id = ci.city_id'''

diamonds = pd.read_sql_query(query,connection)





In [48]:
diamonds.head()

,index_id,clarity,color,cut,depth,table,x,y,z,price,city,carat
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,VS2,J,Premium,62.4,58.0,6.83,6.79,4.25,4268,Dubai,1.21
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,VS2,H,Very Good,63.0,57.0,4.35,4.38,2.75,505,Kimberly,0.32
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,VS1,G,Fair,65.5,55.0,5.62,5.53,3.65,2686,Las Vegas,0.71
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,SI1,D,Good,63.8,56.0,4.68,4.72,3.00,738,Kimberly,0.41
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,SI1,G,Ideal,60.5,59.0,6.55,6.51,3.95,4882,Dubai,1.02


CREAMOS LA COLUMNA DE VOLUMEN PARA PODER ELIMINAR POSTERIORMENTE X,Y,Z

In [49]:
diamonds["volumen"] = diamonds["x"]*diamonds["y"]*diamonds["z"]

In [50]:
diamonds.head()

,index_id,clarity,color,cut,depth,table,x,y,z,price,city,carat,volumen
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,VS2,J,Premium,62.4,58.0,6.83,6.79,4.25,4268,Dubai,1.21,197.096725
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,VS2,H,Very Good,63.0,57.0,4.35,4.38,2.75,505,Kimberly,0.32,52.395750
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,VS1,G,Fair,65.5,55.0,5.62,5.53,3.65,2686,Las Vegas,0.71,113.436890
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,SI1,D,Good,63.8,56.0,4.68,4.72,3.00,738,Kimberly,0.41,66.268800
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,SI1,G,Ideal,60.5,59.0,6.55,6.51,3.95,4882,Dubai,1.02,168.429975


In [51]:
diamonds = diamonds.drop(['x','y','z'], axis = 1)

CREAMOS UNA COLUMNA DE DENSIDAD QUE ES LA VARIBLE QUE MAS CORRELACIONADA ESTA CON EL PRECIO DE LOS DIAMANTES  V/W

In [52]:
diamonds["V/W"] = diamonds["volumen"]/diamonds["carat"]
diamonds.head()

,index_id,clarity,color,cut,depth,table,price,city,carat,volumen,V/W
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,VS2,J,Premium,62.4,58.0,4268,Dubai,1.21,197.096725,162.889855
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,VS2,H,Very Good,63.0,57.0,505,Kimberly,0.32,52.395750,163.736719
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,VS1,G,Fair,65.5,55.0,2686,Las Vegas,0.71,113.436890,159.770268
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,SI1,D,Good,63.8,56.0,738,Kimberly,0.41,66.268800,161.631220
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,SI1,G,Ideal,60.5,59.0,4882,Dubai,1.02,168.429975,165.127426


ELIMINAMOS LOS OUTLIERS

In [53]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.05)
    q3 = df_in[col_name].quantile(0.95)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [54]:
diamonds =remove_outlier(diamonds,"depth")
diamonds =remove_outlier(diamonds,"table")
diamonds=remove_outlier(diamonds,"carat")
diamonds=remove_outlier(diamonds,"volumen")
diamonds=remove_outlier(diamonds,"V/W")

In [55]:
diamonds.head()

,index_id,clarity,color,cut,depth,table,price,city,carat,volumen,V/W
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,VS2,J,Premium,62.4,58.0,4268,Dubai,1.21,197.096725,162.889855
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,VS2,H,Very Good,63.0,57.0,505,Kimberly,0.32,52.395750,163.736719
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,VS1,G,Fair,65.5,55.0,2686,Las Vegas,0.71,113.436890,159.770268
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,SI1,D,Good,63.8,56.0,738,Kimberly,0.41,66.268800,161.631220
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,SI1,G,Ideal,60.5,59.0,4882,Dubai,1.02,168.429975,165.127426


In [56]:
diamonds.describe()

,depth,table,price,carat,volumen,V/W
count,40381.000000,40381.000000,40381.000000,40381.00000,40381.000000,40381.000000
mean,61.750890,57.441997,3924.420495,0.79680,129.682953,163.293807
std,1.406279,2.214663,3987.687303,0.47379,76.494844,3.121923
min,52.700000,44.000000,326.000000,0.20000,32.262375,143.439320
25%,61.000000,56.000000,945.000000,0.40000,64.890126,161.559685
50%,61.800000,57.000000,2395.000000,0.70000,114.711036,163.485000
75%,62.500000,59.000000,5325.000000,1.04000,170.844704,165.347706
max,70.500000,70.000000,18823.000000,3.51000,560.945574,177.108314


In [57]:
diamonds.corr()

,depth,table,price,carat,volumen,V/W
depth,1.000000,-0.303309,-0.015369,0.025722,0.008183,-0.445535
table,-0.303309,1.000000,0.130136,0.183549,0.174406,-0.362225
price,-0.015369,0.130136,1.000000,0.922574,0.924715,-0.200079
carat,0.025722,0.183549,0.922574,1.000000,0.999254,-0.290440
volumen,0.008183,0.174406,0.924715,0.999254,1.000000,-0.258355
V/W,-0.445535,-0.362225,-0.200079,-0.290440,-0.258355,1.000000


como podemos ver en el volumen seguimos teniendo ceros por lo que vamos a limpiarlos manualmente, para asegurarnos que se eliminan

In [58]:

filas_eliminar_volumen = diamonds.loc[diamonds['volumen'] == 0.000000]
labels_diamonds = filas_eliminar_volumen.index.to_list()

In [59]:
diamonds = diamonds.drop(labels= labels_diamonds , axis=0)

In [60]:
#COMO PODEMOS VER YA NO TENEMOS NINGUN VALOR DEL VOLUMEN QUE SEA CERO POR LO QUE NOS HEMOS QUITADO ESOS OUTLIERS
diamonds.describe()

,depth,table,price,carat,volumen,V/W
count,40381.000000,40381.000000,40381.000000,40381.00000,40381.000000,40381.000000
mean,61.750890,57.441997,3924.420495,0.79680,129.682953,163.293807
std,1.406279,2.214663,3987.687303,0.47379,76.494844,3.121923
min,52.700000,44.000000,326.000000,0.20000,32.262375,143.439320
25%,61.000000,56.000000,945.000000,0.40000,64.890126,161.559685
50%,61.800000,57.000000,2395.000000,0.70000,114.711036,163.485000
75%,62.500000,59.000000,5325.000000,1.04000,170.844704,165.347706
max,70.500000,70.000000,18823.000000,3.51000,560.945574,177.108314


ANTES DE CREAR LAS VARIABLES DUMMIES VAMOS A ELIMINAR LA COLUMNA DE index_id YA QUE NOS APORTA MUY POCO

In [61]:
diamonds = diamonds.drop(["index_id"], axis = 1)

AHORA YA TENEMOS TODOS LOS DATOS LIMPIOS PARA PODER CREAR LAS VARIABLES DUMMIES

In [62]:
dummy_clarity = pd.get_dummies(diamonds["clarity"], prefix= "clarity")
dummy_color = pd.get_dummies(diamonds["color"], prefix= "color")
dummy_cut = pd.get_dummies(diamonds["cut"], prefix= "cut")
dummy_city = pd.get_dummies(diamonds["city"], prefix= "city")


diamonds_dummies_cleaned = pd.concat([diamonds, dummy_clarity,dummy_color,dummy_cut,dummy_city], axis=1)
lista_columnas = diamonds_dummies_cleaned.columns.values.tolist()

diamonds_dummies_cleaned

,clarity,color,cut,depth,table,price,city,carat,volumen,V/W,...,city_Kimberly,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,city_Tel Aviv,city_Zurich
0,VS2,J,Premium,62.4,58.0,4268,Dubai,1.21,197.096725,162.889855,...,0,0,0,0,0,0,0,0,0,0
1,VS2,H,Very Good,63.0,57.0,505,Kimberly,0.32,52.395750,163.736719,...,1,0,0,0,0,0,0,0,0,0
2,VS1,G,Fair,65.5,55.0,2686,Las Vegas,0.71,113.436890,159.770268,...,0,1,0,0,0,0,0,0,0,0
3,SI1,D,Good,63.8,56.0,738,Kimberly,0.41,66.268800,161.631220,...,1,0,0,0,0,0,0,0,0,0
4,SI1,G,Ideal,60.5,59.0,4882,Dubai,1.02,168.429975,165.127426,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,VS1,G,Ideal,62.7,57.0,10070,Antwerp,1.34,221.429120,165.245612,...,0,0,0,0,0,0,0,0,0,0
40451,SI2,F,Good,57.1,60.0,12615,Madrid,2.02,324.276975,160.533156,...,0,0,0,0,1,0,0,0,0,0
40452,SI1,H,Ideal,62.7,56.0,5457,Kimberly,1.01,163.990554,162.366885,...,1,0,0,0,0,0,0,0,0,0
40453,VS1,J,Ideal,61.9,54.3,456,Kimberly,0.33,54.900540,166.365273,...,1,0,0,0,0,0,0,0,0,0


In [63]:
diamonds_dummies_cleaned.corr()

,depth,table,price,carat,volumen,V/W,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,...,city_Kimberly,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,city_Tel Aviv,city_Zurich
depth,1.000000,-0.303309,-0.015369,0.025722,0.008183,-0.445535,0.081988,-0.030463,0.043533,0.007317,...,0.003480,0.006262,-0.004757,0.001050,-0.000044,-0.000730,-0.000905,-0.003021,0.007894,-0.003840
table,-0.303309,1.000000,0.130136,0.183549,0.174406,-0.362225,0.041332,-0.078934,0.050908,0.094975,...,0.004159,0.000867,0.005246,0.001251,-0.002789,-0.001110,-0.003213,0.003614,-0.007787,-0.007019
price,-0.015369,0.130136,1.000000,0.922574,0.924715,-0.200079,-0.001411,-0.051998,0.009732,0.133850,...,0.004525,-0.000494,0.001564,-0.004318,0.003203,-0.000115,-0.002024,0.003311,-0.011276,0.006167
carat,0.025722,0.183549,0.922574,1.000000,0.999254,-0.290440,0.119233,-0.115450,0.063743,0.272691,...,0.002934,-0.001604,0.002773,-0.002844,0.002053,0.002795,-0.002391,0.003428,-0.011849,0.004198
volumen,0.008183,0.174406,0.924715,0.999254,1.000000,-0.258355,0.114700,-0.113860,0.062576,0.271370,...,0.002891,-0.001919,0.003032,-0.003015,0.002254,0.002792,-0.002373,0.003250,-0.011823,0.004330
V/W,-0.445535,-0.362225,-0.200079,-0.290440,-0.258355,1.000000,-0.111919,0.118873,-0.076183,-0.118706,...,-0.001973,-0.004134,0.002015,-0.001477,0.006033,0.000959,0.002254,-0.002628,0.001815,0.001201
clarity_I1,0.081988,0.041332,-0.001411,0.119233,0.114700,-0.111919,1.000000,-0.021632,-0.066173,-0.053324,...,-0.006179,0.003406,-0.002162,0.004937,0.004024,0.001645,-0.004718,-0.003113,0.003534,-0.005214
clarity_IF,-0.030463,-0.078934,-0.051998,-0.115450,-0.113860,0.118873,-0.021632,1.000000,-0.103859,-0.083692,...,-0.003387,0.002962,0.004235,0.001504,-0.009844,0.001807,-0.000543,0.010398,-0.001244,-0.011435
clarity_SI1,0.043533,0.050908,0.009732,0.063743,0.062576,-0.076183,-0.066173,-0.103859,1.000000,-0.256014,...,-0.001911,-0.003911,0.005174,0.002919,0.004127,0.004786,-0.007418,-0.012670,-0.000298,0.003227
clarity_SI2,0.007317,0.094975,0.133850,0.272691,0.271370,-0.118706,-0.053324,-0.083692,-0.256014,1.000000,...,-0.001592,0.001189,-0.003403,-0.001738,-0.000867,0.003562,0.001630,0.006215,-0.005155,0.001090


ELIMINAMOS LAS COLUMNAS DE LAS QUE HEMOS SACADO LAS COLUMNAS DUMMIES

In [64]:
diamonds_dummies_cleaned = diamonds_dummies_cleaned.drop(['clarity','color','cut','city'], axis = 1)

In [65]:
diamonds_dummies_cleaned.corr()

,depth,table,price,carat,volumen,V/W,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,...,city_Kimberly,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,city_Tel Aviv,city_Zurich
depth,1.000000,-0.303309,-0.015369,0.025722,0.008183,-0.445535,0.081988,-0.030463,0.043533,0.007317,...,0.003480,0.006262,-0.004757,0.001050,-0.000044,-0.000730,-0.000905,-0.003021,0.007894,-0.003840
table,-0.303309,1.000000,0.130136,0.183549,0.174406,-0.362225,0.041332,-0.078934,0.050908,0.094975,...,0.004159,0.000867,0.005246,0.001251,-0.002789,-0.001110,-0.003213,0.003614,-0.007787,-0.007019
price,-0.015369,0.130136,1.000000,0.922574,0.924715,-0.200079,-0.001411,-0.051998,0.009732,0.133850,...,0.004525,-0.000494,0.001564,-0.004318,0.003203,-0.000115,-0.002024,0.003311,-0.011276,0.006167
carat,0.025722,0.183549,0.922574,1.000000,0.999254,-0.290440,0.119233,-0.115450,0.063743,0.272691,...,0.002934,-0.001604,0.002773,-0.002844,0.002053,0.002795,-0.002391,0.003428,-0.011849,0.004198
volumen,0.008183,0.174406,0.924715,0.999254,1.000000,-0.258355,0.114700,-0.113860,0.062576,0.271370,...,0.002891,-0.001919,0.003032,-0.003015,0.002254,0.002792,-0.002373,0.003250,-0.011823,0.004330
V/W,-0.445535,-0.362225,-0.200079,-0.290440,-0.258355,1.000000,-0.111919,0.118873,-0.076183,-0.118706,...,-0.001973,-0.004134,0.002015,-0.001477,0.006033,0.000959,0.002254,-0.002628,0.001815,0.001201
clarity_I1,0.081988,0.041332,-0.001411,0.119233,0.114700,-0.111919,1.000000,-0.021632,-0.066173,-0.053324,...,-0.006179,0.003406,-0.002162,0.004937,0.004024,0.001645,-0.004718,-0.003113,0.003534,-0.005214
clarity_IF,-0.030463,-0.078934,-0.051998,-0.115450,-0.113860,0.118873,-0.021632,1.000000,-0.103859,-0.083692,...,-0.003387,0.002962,0.004235,0.001504,-0.009844,0.001807,-0.000543,0.010398,-0.001244,-0.011435
clarity_SI1,0.043533,0.050908,0.009732,0.063743,0.062576,-0.076183,-0.066173,-0.103859,1.000000,-0.256014,...,-0.001911,-0.003911,0.005174,0.002919,0.004127,0.004786,-0.007418,-0.012670,-0.000298,0.003227
clarity_SI2,0.007317,0.094975,0.133850,0.272691,0.271370,-0.118706,-0.053324,-0.083692,-0.256014,1.000000,...,-0.001592,0.001189,-0.003403,-0.001738,-0.000867,0.003562,0.001630,0.006215,-0.005155,0.001090


HACEMOS UN LISTADO CON NUESTRAS FEATURES

In [66]:
features = [
 'volumen',
 "V/W",
 'carat',
 'clarity_I1',
 'clarity_IF',
 'clarity_SI1',
 'clarity_SI2',
 'clarity_VS1',
 'clarity_VS2',
 'clarity_VVS1',
 'clarity_VVS2',
 'color_D',
 'color_E',
 'color_F',
 'color_G',
 'color_H',
 'color_I',
 'color_J',
 'cut_Fair',
 'cut_Good',
 'cut_Ideal',
 'cut_Premium',
 'cut_Very Good',
 ]

VAMOS A APLICAR EL ROBUST SCALER

In [67]:
scaler = RobustScaler()
X = scaler.fit_transform(diamonds_dummies_cleaned[features])
y = diamonds_dummies_cleaned["price"]

CREAMOS EL MODELO

In [68]:
 X_train, X_validacion, y_train, y_validacion = train_test_split(X, y, test_size=0.2, random_state=42)

In [69]:
print(f"X_train: {X_train.shape}, X_test: {X_validacion.shape}, y_train: {y_train.shape}")

X_train: (32304, 23), X_test: (8077, 23), y_train: (32304,)


In [70]:
#MODEL

#regressor = LinearRegression()
regressor = RandomForestRegressor(random_state= 42)
hyperparameters = regressor.get_params()
regressor.fit(X_train, y_train)
y_pred_train = regressor.predict(X_validacion)

print('Model:', regressor, '\n')
print('Model hyperparameters:', hyperparameters, '\n')
print('Ground truth target:', y_validacion, '\n')
print('Predicted target:', y_pred_train, '\n')

Model: RandomForestRegressor(random_state=42) 

Model hyperparameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False} 

Ground truth target: 27535    1094
6063     6459
25995     926
26873    1595
31207     658
         ... 
39477    1080
22549    2511
1724      756
14539     447
16390     666
Name: price, Length: 8077, dtype: int64 

Predicted target: [1067.23       6720.005       885.98633333 ...  640.06        491.443
  790.41      ] 



MIRAMOS EL RMSE(ERROR) QUE TENEMOS EN NUESTRO MODELO

In [71]:
RMSE_ERROR_sklearn = mean_squared_error(y_validacion,y_pred_train, squared = False)
print("TENEMOS UN ERROR DEL " + str(RMSE_ERROR_sklearn) + "%")

TENEMOS UN ERROR DEL 562.6933589862581%


LEEMOS EL DATASET DE TEST 

In [72]:
diamonds_test = pd.read_csv("../KAGGLE_PRUBAS_DATA/DATA/diamonds_test.csv")

CREAMOS LA COLUMNA DE VOLUMEN


In [73]:
diamonds_test["volumen"] = diamonds_test["x"]*diamonds_test["y"]*diamonds_test["z"]

In [74]:
diamonds_test = diamonds_test.drop(['x','y','z'], axis = 1)

In [75]:
diamonds_test

,id,carat,cut,color,clarity,depth,table,city,volumen
0,0,0.79,Very Good,F,SI1,62.7,60.0,Amsterdam,125.806866
1,1,1.20,Ideal,J,VS1,61.0,57.0,Surat,196.129362
2,2,1.57,Premium,H,SI1,62.2,61.0,Kimberly,246.878712
3,3,0.90,Very Good,F,SI1,63.8,54.0,Kimberly,145.593630
4,4,0.50,Very Good,F,VS1,62.9,58.0,Amsterdam,81.997355
...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,Amsterdam,93.924600
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,New York City,116.477148
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,Tel Aviv,115.898725
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,Surat,118.874925


CREAMOS LA COLUMNA DE DENSIDAD PARA EL DATASET DE TEST

In [76]:
diamonds_test["V/W"] = diamonds_test["volumen"]/diamonds_test["carat"]

In [77]:
diamonds_test.head()

,id,carat,cut,color,clarity,depth,table,city,volumen,V/W
0,0,0.79,Very Good,F,SI1,62.7,60.0,Amsterdam,125.806866,159.249197
1,1,1.20,Ideal,J,VS1,61.0,57.0,Surat,196.129362,163.441135
2,2,1.57,Premium,H,SI1,62.2,61.0,Kimberly,246.878712,157.247587
3,3,0.90,Very Good,F,SI1,63.8,54.0,Kimberly,145.593630,161.770700
4,4,0.50,Very Good,F,VS1,62.9,58.0,Amsterdam,81.997355,163.994710


CREAMOS LAS VARIABLES DUMMIES DEL DATASET DE TEST

In [78]:
dummy_clarity_test = pd.get_dummies(diamonds_test["clarity"], prefix= "clarity")
dummy_color_test = pd.get_dummies(diamonds_test["color"], prefix= "color")
dummy_cut_test = pd.get_dummies(diamonds_test["cut"], prefix= "cut")
dummy_city_test = pd.get_dummies(diamonds_test["city"], prefix= "city")


diamonds_test_cleaned = pd.concat([diamonds_test, dummy_clarity_test, dummy_color_test, dummy_cut_test, dummy_city_test], axis=1)
lista_columnas_test = diamonds_test_cleaned.columns.values.tolist()

diamonds_test_cleaned

,id,carat,cut,color,clarity,depth,table,city,volumen,V/W,...,city_Kimberly,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,city_Tel Aviv,city_Zurich
0,0,0.79,Very Good,F,SI1,62.7,60.0,Amsterdam,125.806866,159.249197,...,0,0,0,0,0,0,0,0,0,0
1,1,1.20,Ideal,J,VS1,61.0,57.0,Surat,196.129362,163.441135,...,0,0,0,0,0,0,0,1,0,0
2,2,1.57,Premium,H,SI1,62.2,61.0,Kimberly,246.878712,157.247587,...,1,0,0,0,0,0,0,0,0,0
3,3,0.90,Very Good,F,SI1,63.8,54.0,Kimberly,145.593630,161.770700,...,1,0,0,0,0,0,0,0,0,0
4,4,0.50,Very Good,F,VS1,62.9,58.0,Amsterdam,81.997355,163.994710,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,Amsterdam,93.924600,164.780000,...,0,0,0,0,0,0,0,0,0,0
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,New York City,116.477148,164.052321,...,0,0,0,0,0,1,0,0,0,0
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,Tel Aviv,115.898725,165.569607,...,0,0,0,0,0,0,0,0,1,0
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,Surat,118.874925,169.821321,...,0,0,0,0,0,0,0,1,0,0


ELIMINAMOS LAS COLUMNAS DE LAS QUE HEMOS SACADO LAS VARIABLES DUMMIES

In [79]:
diamonds_test_cleaned = diamonds_test_cleaned.drop(['clarity','color','cut','city'], axis = 1)

LE APLICAMOS EL SCALER AL TEST 

In [80]:
scaler = RobustScaler()
X_test2 = scaler.fit_transform(diamonds_test_cleaned[features])


REALIZAMOS LA PREDICTION

In [81]:
y_pred = regressor.predict(X_test2).clip(0,30000)

print('Model:', regressor, '\n')
print('Model hyperparameters:', hyperparameters, '\n')

Model: RandomForestRegressor(random_state=42) 

Model hyperparameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False} 



CREAMOS LA SUBMISSION

In [82]:
submission = pd.DataFrame({'id': diamonds_test_cleaned["id"] , 'price': y_pred})
submission.shape

(13485, 2)

In [83]:
os.listdir("../KAGGLE_PRUBAS_DATA/DATA")

['diamonds_test.csv',
 'submission4.csv',
 'submission.csv',
 'diamonds_train.db',
 'submission2.csv',
 'submission3.csv',
 'sample_submission.csv']

In [84]:
submission.to_csv("../KAGGLE_PRUBAS_DATA/DATA/submission4.csv", index=False)